# 한빛출판네트워크 새로나온 책 Crawling

In [1]:
from bs4 import BeautifulSoup 
import urllib.request
import pandas as pd 

In [2]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
url = base_url + sub_url
with urllib.request.urlopen(url) as response:
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')

In [3]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [4]:
lis[0]

<li class="sub_book_list">
<div class="view_box">
<span class="view_box_block">
<span class="img_hidden_a">
<span class="pop_quick_menu">
<a href="/academy/books/book_view.html?p_code=B4797877052"><img alt="상세보기" src="https://www.hanbit.co.kr/images/common/icon_view_detail.png"/></a>
<a href="javascript:;" onclick="addCart('B4797877052','','1','');">
<img alt="장바구니 담기" src="https://www.hanbit.co.kr/images/common/icon_add_cart.png"/>
</a>
</span>
<span class="price">32,000<span>원</span></span>
<span class="pop_quick_bg" onclick="location='/academy/books/book_view.html?p_code=B4797877052'"></span>
</span>
<!-- 예약도서인 경우 -->
<!-- ebook이 도서인 경우 -->
<img alt="" class="thumb" src="/data/books/B4797877052_m.jpg"/>
</span>
<div class="info">
<p class="book_tit"><a href="/academy/books/book_view.html?p_code=B4797877052">포르잔 C++ 에센셜</a></p>
<p class="book_writer">Behrouz A. Forouzan , Richard F. Gilberg </p>
</div>
</div>
</li>

In [5]:
book_url = lis[0].find('a').attrs['href']
book_url

'/academy/books/book_view.html?p_code=B4797877052'

In [6]:
with urllib.request.urlopen(base_url+book_url) as response:
    html = response.read()
    book_soup = BeautifulSoup(html, 'html.parser')

In [7]:
info = book_soup.select_one('.store_product_info_box')
title = info.find('h3').string
title

'포르잔 C++ 에센셜'

In [8]:
lis = info.find_all('li')
book_dict = {}
for li in lis:
    idx = li.find('strong').get_text().find(':')
    key = li.find('strong').get_text()[:idx-1]
    value = li.find('span').get_text()
    book_dict.update([[key, value]])
book_dict

{'저자': 'Behrouz A. Forouzan , Richard F. Gilberg ',
 '번역': '윤인성 ',
 '출간': '2020-11-10',
 '페이지': '756 쪽',
 'ISBN': '9791156645061',
 '물류코드': '4506'}

In [9]:
al = 'Behrouz A. Forouzan , Richard F. Gilberg '.split(',')
al = list(map(lambda s: s.rstrip(), al))
','.join(al)

'Behrouz A. Forouzan, Richard F. Gilberg'

In [10]:
info = book_soup.select_one('.store_product_info_box')
book_lis = info.find_all('li')
al = book_lis[0].find('span').get_text().split(',')
authors = ','.join(list(map(lambda s: s.rstrip(), al)))

for bli in book_lis[1:]:
    key = bli.find('strong').get_text()
    if key.find('번역') >= 0:
        tl = bli.find('span').get_text().split(',')
        translators = ','.join(list(map(lambda s: s.rstrip(), tl)))
    elif key.find('출간') >= 0:
        date = bli.find('span').get_text()
    elif key.find('페이지') >= 0:
        page = int(bli.find('span').get_text()[:-2])
    elif key.find('ISBN') >= 0:
        isbn = bli.find('span').get_text()

print(authors, translators, date, page, isbn)

Behrouz A. Forouzan, Richard F. Gilberg 윤인성 2020-11-10 756 9791156645061


In [11]:
pb_ps = book_soup.select_one('.payment_box').find_all('p')
list_prices = []; sales_prices = []; mileages = []

idx = pb_ps[0].select_one('.pbr').get_text().find('원')
lp = pb_ps[0].select_one('.pbr').get_text()[:idx]
lp = int(lp.replace(',',''))

idx = pb_ps[1].select_one('.pbr').get_text().find('원')
sp = pb_ps[1].select_one('.pbr').get_text()[:idx]
sp = int(sp.replace(',',''))

idx = pb_ps[2].select_one('.pbr').get_text().find('점')
mile = pb_ps[2].select_one('.pbr').get_text()[:idx]
mile = int(mile.replace(',',''))

print(lp, sp, mile)

32000 32000 960


#### 첫번째 페이지

In [13]:
title_list = []; info_list = []; pb_list = []
lis = soup.select('.sub_book_list')
for li in lis:
    book_url = li.find('a').attrs['href']
    with urllib.request.urlopen(base_url+book_url) as response:
        html = response.read()
        book_soup = BeautifulSoup(html, 'html.parser')
    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').string
    title_list.append(title)

    book_lis = info.find_all('li')
    book_dict = {}
    for book_li in book_lis:
        idx = book_li.find('strong').get_text().find(':')
        key = book_li.find('strong').get_text()[:idx-1]
        value = book_li.find('span').get_text()
        book_dict.update([[key, value]])
    info_list.append(book_dict)

    pb_ps = book_soup.select_one('.payment_box').find_all('p')
    pb_dict = {}
    for pb in pb_ps:
        idx = pb.select_one('.pbl').get_text().find(':')
        key = pb.select_one('.pbl').get_text()[:idx-1]
        value = pb.select_one('.pbr').get_text()
        pb_dict.update([[key, value]])
    pb_list.append(pb_dict)

In [14]:
print(title_list)

['포르잔 C++ 에센셜', 'IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습', '포르잔 C++ 바이블', '딥러닝을 위한 선형대수학', 'IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)', '대학을 버려야 대학이 산다', 'IT CookBook, 따라 하면서 배우는 사물인터넷', 'R로 하는 다변량 데이터 분석 : 이론부터 실무 활용까지', '창의적 공학설계(3판)', '공학 핵심수학(2판)  : 기초부터 심화까지 핵심만 쏙쏙', '응용이 보이는 선형대수학 : 파이썬과 함께하는 선형대수학 이론과 응용', 'IT CookBook, 초연결 사회의 데이터통신과 네트워킹', 'IT CookBook, 쉽게 배우는 MFC 윈도우 프로그래밍', 'IT CookBook, 인공지능 시대를 위한 컴퓨터 과학 개론', 'IT CookBook, 코틀린을 활용한 안드로이드 프로그래밍', 'IT CookBook, 처음 만나는 신재생에너지', '프리드버그 선형대수학', 'IT CookBook, CentOS 리눅스', 'Microeconomics, 6th Edition, Asia Edition', '누구나 파이썬 통계분석']


In [15]:
info_list

[{'저자': 'Behrouz A. Forouzan , Richard F. Gilberg ',
  '번역': '윤인성 ',
  '출간': '2020-11-10',
  '페이지': '756 쪽',
  'ISBN': '9791156645061',
  '물류코드': '4506'},
 {'저자': '김지연 ',
  '출간': '2020-09-28',
  '페이지': '536 쪽',
  'ISBN': '9791156645054',
  '물류코드': '4505',
  '구판정보': '이 도서는 <IT CookBook, 데이터베이스 기초와 실습 : 액세스 2013>의 개정판입니다. 구판 정보 보기'},
 {'저자': 'Behrouz A. Forouzan , Richard F. Gilberg ',
  '번역': '윤인성 ',
  '출간': '2020-08-31',
  '페이지': '1192 쪽',
  'ISBN': '9791156645030',
  '물류코드': '4503'},
 {'저자': 'Gilbert Strang ',
  '번역': '부산대학교 산업수학센터(FFMIMC) ',
  '출간': '2020-08-05',
  '페이지': '560 쪽',
  'ISBN': '9791156644972',
  '물류코드': '4497'},
 {'저자': '박우창 , 남송휘 , 이현룡 ',
  '출간': '2020-07-30',
  '페이지': '600 쪽',
  'ISBN': '9791156645023',
  '물류코드': '4502',
  '구판정보': '이 도서는 <IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습>의 개정판입니다. 구판 정보 보기'},
 {'저자': '조동성 외 인천대학교 가족 178명 ',
  '출간': '2020-07-25',
  '페이지': '216 쪽',
  'ISBN': '9791156645016',
  '물류코드': '4501'},
 {'저자': '양재삼 ',
  '출간': '2020-07-23',
  '페이지': '448 쪽',
 

In [16]:
pb_list

[{'정가': '32,000원', '판매가': '32,000원(0% off)', '마일리지': '960점 (3%)'},
 {'정가': '28,000원', '판매가': '28,000원(0% off)', '마일리지': '840점 (3%)'},
 {'정가': '45,000원', '판매가': '45,000원(0% off)', '마일리지': '1,350점 (3%)'},
 {'정가': '38,000원', '판매가': '38,000원(0% off)', '마일리지': '1,140점 (3%)'},
 {'정가': '29,000원', '판매가': '29,000원(0% off)', '마일리지': '870점 (3%)'},
 {'정가': '18,000원', '판매가': '18,000원(0% off)', '마일리지': '540점 (3%)'},
 {'정가': '28,000원', '판매가': '28,000원(0% off)', '마일리지': '840점 (3%)'},
 {'정가': '32,000원', '판매가': '32,000원(0% off)', '마일리지': '960점 (3%)'},
 {'정가': '23,000원', '판매가': '23,000원(0% off)', '마일리지': '690점 (3%)'},
 {'정가': '23,000원', '판매가': '23,000원(0% off)', '마일리지': '690점 (3%)'},
 {'정가': '31,000원', '판매가': '31,000원(0% off)', '마일리지': '930점 (3%)'},
 {'정가': '29,000원', '판매가': '29,000원(0% off)', '마일리지': '870점 (3%)'},
 {'정가': '30,000원', '판매가': '30,000원(0% off)', '마일리지': '900점 (3%)'},
 {'정가': '27,000원', '판매가': '27,000원(0% off)', '마일리지': '810점 (3%)'},
 {'정가': '34,000원', '판매가': '34,000원(0% off)', '마일리지': '1,02

In [19]:
title_list = []; author_list = []; translator_list = []
date_list = []; page_list = []; isbn_list = []
list_price_list = []; sales_price_list = []; mileage_list = []

for li in lis:
    book_url = li.find('a').attrs['href']
    with urllib.request.urlopen(base_url+book_url) as response:
        html = response.read()
        book_soup = BeautifulSoup(html, 'html.parser')
    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').string
    title_list.append(title)

    book_lis = info.find_all('li')
    al = book_lis[0].find('span').get_text().split(',')
    authors = ','.join(list(map(lambda s: s.rstrip(), al)))
    author_list.append(authors)

    tr_flag = False
    for bli in book_lis[1:]:
        key = bli.find('strong').get_text()
        if key.find('출간') >= 0:
            date = bli.find('span').get_text()
            date_list.append(date)
            continue
        elif key.find('페이지') >= 0:
            page = int(bli.find('span').get_text()[:-2])
            page_list.append(page)
            continue
        elif key.find('ISBN') >= 0:
            isbn = bli.find('span').get_text()
            isbn_list.append(isbn)
            continue
        elif key.find('번역') >= 0:
            tl = bli.find('span').get_text().split(',')
            translators = ','.join(list(map(lambda s: s.rstrip(), tl)))
            translator_list.append(translators)
            tr_flag = True
    
    if not tr_flag:
        translator_list.append(' ')

    pb_ps = book_soup.select_one('.payment_box').find_all('p')
    idx = pb_ps[0].select_one('.pbr').get_text().find('원')
    lp = pb_ps[0].select_one('.pbr').get_text()[:idx]
    lp = int(lp.replace(',',''))
    list_price_list.append(lp)

    idx = pb_ps[1].select_one('.pbr').get_text().find('원')
    sp = pb_ps[1].select_one('.pbr').get_text()[:idx]
    sp = int(sp.replace(',',''))
    sales_price_list.append(sp)

    idx = pb_ps[2].select_one('.pbr').get_text().find('점')
    mile = pb_ps[2].select_one('.pbr').get_text()[:idx]
    mile = int(mile.replace(',',''))
    mileage_list.append(mile)

In [20]:
len(title_list), len(author_list), len(translator_list), len(date_list), len(page_list), len(isbn_list), len(list_price_list), len(sales_price_list), len(mileage_list)

(20, 20, 20, 20, 20, 20, 20, 20, 20)

In [21]:
new_book = pd.DataFrame({
            '제목': title_list,
            '저자': author_list,
            '번역': translator_list,
            '출간일': date_list,
            '페이지': page_list,
            'ISBN': isbn_list,
            '정가': list_price_list,
            '판가': sales_price_list,
            '마일리지': mileage_list
})
new_book

,제목,저자,번역,출간일,페이지,ISBN,정가,판가,마일리지
0,포르잔 C++ 에센셜,"Behrouz A. Forouzan, Richard F. Gilberg",윤인성,2020-11-10,756,9791156645061,32000,32000,960
1,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",김지연,,2020-09-28,536,9791156645054,28000,28000,840
2,포르잔 C++ 바이블,"Behrouz A. Forouzan, Richard F. Gilberg",윤인성,2020-08-31,1192,9791156645030,45000,45000,1350
3,딥러닝을 위한 선형대수학,Gilbert Strang,부산대학교 산업수학센터(FFMIMC),2020-08-05,560,9791156644972,38000,38000,1140
4,"IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)","박우창, 남송휘, 이현룡",,2020-07-30,600,9791156645023,29000,29000,870
5,대학을 버려야 대학이 산다,조동성 외 인천대학교 가족 178명,,2020-07-25,216,9791156645016,18000,18000,540
6,"IT CookBook, 따라 하면서 배우는 사물인터넷",양재삼,,2020-07-23,448,9791156644941,28000,28000,840
7,R로 하는 다변량 데이터 분석 : 이론부터 실무 활용까지,"카와하시 잇코, 이와마 노리카즈, 스즈키 마사유키",권기태,2020-07-10,544,9791156644989,32000,32000,960
8,창의적 공학설계(3판),김은경,,2020-07-10,408,9791156644934,23000,23000,690
9,공학 핵심수학(2판) : 기초부터 심화까지 핵심만 쏙쏙,함남우,,2020-07-06,484,9791156644996,23000,23000,690


### 모든 페이지에 대해서 처리

In [22]:
from tqdm import tqdm_notebook

In [23]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
title_list = []; author_list = []; translator_list = []
date_list = []; page_list = []; isbn_list = []
list_price_list = []; sales_price_list = []; mileage_list = []
total_page = 16

In [24]:
for page in tqdm_notebook(range(total_page)):
    page_url = f'?page={page+1}&cate_cd=&srt=&searchKey=&keyWord='
    url = base_url + sub_url + page_url
    with urllib.request.urlopen(url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')

    lis = soup.select('.sub_book_list')
    for li in lis:
        book_url = li.find('a').attrs['href']
        with urllib.request.urlopen(base_url+book_url) as response:
            html = response.read()
            book_soup = BeautifulSoup(html, 'html.parser')
        info = book_soup.select_one('.store_product_info_box')
        title = info.find('h3').string
        title_list.append(title)

        book_lis = info.find_all('li')
        al = book_lis[0].find('span').get_text().split(',')
        authors = ','.join(list(map(lambda s: s.rstrip(), al)))
        author_list.append(authors)

        tr_flag = False
        for bli in book_lis[1:]:
            key = bli.find('strong').get_text()
            if key.find('출간') >= 0:
                date = bli.find('span').get_text()
                date_list.append(date)
                continue
            elif key.find('페이지') >= 0:
                page = int(bli.find('span').get_text()[:-2])
                page_list.append(page)
                continue
            elif key.find('ISBN') >= 0:
                isbn = bli.find('span').get_text()
                isbn_list.append(isbn)
                continue
            elif key.find('번역') >= 0:
                tl = bli.find('span').get_text().split(',')
                translators = ','.join(list(map(lambda s: s.rstrip(), tl)))
                translator_list.append(translators)
                tr_flag = True
        
        if not tr_flag:
            translator_list.append(' ')

        pb_ps = book_soup.select_one('.payment_box').find_all('p')
        idx = pb_ps[0].select_one('.pbr').get_text().find('원')
        lp = pb_ps[0].select_one('.pbr').get_text()[:idx]
        lp = int(lp.replace(',',''))
        list_price_list.append(lp)

        idx = pb_ps[1].select_one('.pbr').get_text().find('원')
        sp = pb_ps[1].select_one('.pbr').get_text()[:idx]
        sp = int(sp.replace(',',''))
        sales_price_list.append(sp)

        idx = pb_ps[2].select_one('.pbr').get_text().find('점')
        mile = pb_ps[2].select_one('.pbr').get_text()[:idx]
        mile = int(mile.replace(',',''))
        mileage_list.append(mile)

In [25]:
new_book = pd.DataFrame({
            '제목': title_list,
            '저자': author_list,
            '번역': translator_list,
            '출간일': date_list,
            '페이지': page_list,
            'ISBN': isbn_list,
            '정가': list_price_list,
            '판가': sales_price_list,
            '마일리지': mileage_list
})
new_book.tail()

,제목,저자,번역,출간일,페이지,ISBN,정가,판가,마일리지
296,문제해결력을 키우는 이공계 글쓰기 : 발표와 토론까지,한성우,,2013-02-28,396,9788998756024,18000,18000,540
297,"진로개발과 직업세계 : 대학생과 청년층을 위한 진로, 취업지침서","서일수, 김도균, 김지혜, 이정배",,2013-02-28,392,9788998756048,20000,20000,600
298,Essential Calculus,James Stewart,,2012-07-08,960,9781133492566,36000,36000,1080
299,"IT CookBook, 디지털 통신 시스템 : Digital Communicatio...","정호영, 임승각",,2011-01-31,576,9788979147988,25000,25000,750
300,"IT CookBook, 운영체제론 : Operating Systems, 3rd Ed","하비 디텔, 폴 디텔, 데이빗 쇼픈스",송경희,2009-09-04,860,9788979146844,32000,32000,960


In [26]:
new_book.to_csv('New_Book_List.csv', sep=',', encoding='utf8')